In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [32]:
df_final = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_final.csv')
df_raw = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_RAW.csv')
col_raw = ["LOC", "IDCODE", "CNRSEQ", "Class", "Gender", "Age", "DIFF", "TMRSZ", "LYMND"
            , "AJCCstage", "BMI_label", "MAGN", "RTDATE", "STDATE", "DEFISURPRIDAT" , "CIG"
            , "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6", "FullDate"    
            ]
col_final = ["LOC", "IDCODE", "CNRSEQ", "Class", "Gender", "Age", "DIFF", "TMRSZ", "LYMND"
            , "AJCCstage", "BMI_label", "MAGN", "RTDATE", "STDATE", "OP" , "CIG"
            , "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6", "FullDate"    
            ]
df_raw = df_raw.loc[(df_raw['CLASSOFCASE'] == 1) | (df_raw['CLASSOFCASE'] == 2)]
df_raw = df_raw[col_raw]
df_final = df_final[col_final]
df_raw['Class'] = df_raw['Class'].apply(lambda x:0 if x == 0 else 1)
df_final['Class'] = df_final['Class'].apply(lambda x:0 if x == 0 else 1)

In [33]:
df_raw['DEFISURPRIDAT'] = df_raw['DEFISURPRIDAT'].fillna(1)
df_raw['OP'] = df_raw['DEFISURPRIDAT'].apply(lambda x:2 if x != 1 else 1)
df_raw = df_raw.drop(columns=['DEFISURPRIDAT'])

In [25]:
df_final['year'] = df_final['FullDate'].apply(lambda x : int(x[:4]))
df_raw['year'] = df_raw['FullDate'].apply(lambda x : int(x[:4]))
df_raw = df_raw.loc[(df_raw.year >= 2011) & (df_raw.CNRSEQ == 1)]
df_final = df_final.loc[(df_final.year >= 2011) & (df_final.CNRSEQ == 1)]
df_raw.to_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_RAW_Process.csv')

### 新四家醫院我編的

In [2]:
fpc = pd.read_csv('/home/refu0917/lungcancer/server/fp_cancer.csv')
spc = pd.read_csv('/home/refu0917/lungcancer/server/sp_cancer.csv')
columns = ['醫院別', '歸戶代號', '有無二癌', '性別', '診斷年齡', '分級分化','病理分級分化','臨床分級分化', '腫瘤大小', '區域淋巴結侵犯數目', '臨床期別組合'
        , '原發部位手術邊緣', '放射治療開始日期', '全身性治療開始日期', '原發部位最確切的手術切除日期', '身高'
        , '體重', '吸菸行為' ,'嚼檳榔行為', '喝酒行為', '癌症部位特定因子1', '癌症部位特定因子2'
        , '癌症部位特定因子3', '癌症部位特定因子4', '癌症部位特定因子6', '最初診斷日期', '個案分類', '序']

col_map = ["LOC", "id","Class", "Gender", "Age", "DIFF","DIFF_path","DIFF_clin", "TMRSZ", "LYMND"
            , "AJCCstage", "MAGN", "RTDATE", "STDATE", "OP", "Height", "Weight"
            , "CIG", "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6", "FullDate", 'Classofcase', 'Seq'    
            ]
fpc = fpc[columns]
spc = spc[columns]
fpc = fpc.rename(columns=dict((i,j)for i,j in zip(columns, col_map)))
spc = spc.rename(columns=dict((i,j)for i,j in zip(columns, col_map)))

spc = spc[spc.Class == 'Y']
df_all = pd.concat([fpc, spc])
df_all = df_all[df_all.LOC != '長庚']
df_all['DIFF_path'] = df_all['DIFF_path'].replace('D', 9)
df_all['DIFF_check'] = df_all['DIFF']

/tmp/ipykernel_3463242/2268356745.py:1: DtypeWarning: Columns (21,120) have mixed types. Specify dtype option on import or set low_memory=False.
  fpc = pd.read_csv('/home/refu0917/lungcancer/server/fp_cancer.csv')


In [3]:
def diff_encode(x):
    if x['DIFF_path'] == 9 or x['DIFF_clin'] == 9: 
        return x[['DIFF_path', 'DIFF_clin']].min()
    else :
        return x[['DIFF_path', 'DIFF_clin']].max()

temp = df_all[df_all.DIFF.isna()]
temp['DIFF_path'] = temp['DIFF_path'].astype(int)
temp['DIFF_clin'] = temp['DIFF_clin'].astype(int)
#temp['DIFF'] = temp.apply(lambda x:x['DIFF_path'] if x['DIFF_path'] < x['DIFF_clin'] else x['DIFF_clin'], axis=1)
temp['DIFF'] = temp.apply(lambda x:diff_encode(x), axis=1)
df_all.loc[df_all.DIFF.isna(), 'DIFF'] = temp['DIFF']
#df_all = df_all.drop(columns = ['DIFF_path', 'DIFF_clin'])
df_all['DIFF'] = df_all['DIFF'].astype(int)

/tmp/ipykernel_3463242/1109803623.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DIFF_path'] = temp['DIFF_path'].astype(int)
/tmp/ipykernel_3463242/1109803623.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['DIFF_clin'] = temp['DIFF_clin'].astype(int)
/tmp/ipykernel_3463242/1109803623.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [4]:
def age_encode(age):
    if age < 21:
        return 1
    elif 21 <= age < 31:
        return 2
    elif 31 <= age < 41:
        return 3
    elif 41 <= age < 51:
        return 4
    elif 51 <= age < 61:
        return 5
    elif 61 <= age < 71:
        return 6
    elif 71 <= age < 81:
        return 7
    elif 81 <= age < 91:
        return 8
    elif age >= 91:
        return 9
def size_encode(size):
    if 1<=size<=49 or 991 <= size <= 995:
        return 1
    elif 50 <= size <= 99:
        return 2
    elif 100 <= size <= 149:
        return 3
    elif 150 <= size < 990:
        return 4
    elif size in [990,998,999]:
        return 9
    else:return np.nan
def lymnd_encode(x):
    if x == 0:
        return 1
    elif 1 <= x <= 2:
        return 2
    elif 3 <= x <= 6:
        return 3
    elif 7 <= x <= 15:
        return 4
    elif 16 <= x < 95:
        return 5
    elif x in [95,97,98,99]:
        return 9
def ajcc_encode(x):
    
    if x in ['BBB', '888', '999', 'OC ']:
        return 9
    else:
        temp = int(x[0])
        return temp+1

def magn_encode(x):
    if x in ['A','B','C']:
        return 2
    else:
        x = int(x)
        if x == 0:
            return 1
        elif 0 < x <7:
            return 2
        elif x in[7,8,9]:
            return 9
        else: return np.nan
    
def date_encode(date):
    if date == 0:
        return 1
    else: 
        return 2

def cig_encode(cig):
    if cig == 88:
        return 1
    elif cig == 999999:
        return 9
    else:
        return 2
def alc_encode(alc):
    if alc == 0:
        return 1
    elif alc == 999:
        return 9
    else: return 2
def bn_encode(bn):
    if bn == 88:
        return 1
    elif bn == 999999:
        return 9
    else: return 2

def bmi(x):
    if x['Height'] == 999 or x['Weight'] == 999:
        return 9
    else:
        h = x['Height']
        h /= 100
        w = x['Weight']
        output = w/(h**2)
        if output < 18.5:
            return 1
        elif 18.5 <= output < 24:
            return 2
        elif output >= 24:
            return 3

def ssf1_encode(x):
    if x == 0 or x == '000':
        return 1
    elif x in[10,20,30,40,'030','010','020','040']:
        return 2
    elif x in [999, 988,'988','999']:
        return 9

def ssf2_encode(x):
    if x == 0 or x == '000':
        return 1
    elif x in[10,20,30,40,'010','020','030','040']:
        return 2
    elif x in [3,999, 988, '988', '999']:
        return 9
    else: return x

def ssf3_encode(x):
    if x in [999,988,998]:
        return 9
    else: return x+1 

def ssf4_encode(x):
    if x == 0:
        return 1
    elif 11<=x<=15:
        return 2
    elif x in [999,988]:
        return 9
    else: return np.nan
    
def ssf6_encode(x):
    if x[0] == '0' or x[0] == '9':
        if x[1] == '2':
            return 1
        elif x[1] == '1':
            return 2
        else: return 9
    else:
        if x[0]  == 'X':
            return 1
        elif x[0] == 'Z':
            return 9
        else: return 2


    
    

In [5]:
df_all['Class'] = df_all['Class'].apply(lambda x:0 if x == 'N' else 1)
df_all['Age'] = df_all['Age'].apply(lambda x:age_encode(x))
df_all['TMRSZ'] = df_all['TMRSZ'].apply(lambda x:size_encode(x))
df_all['LYMND'] = df_all['LYMND'].apply(lambda x:lymnd_encode(x))
df_all['AJCCstage'] = df_all['AJCCstage'].apply(lambda x:ajcc_encode(x))
df_all['MAGN'] = df_all['MAGN'].apply(lambda x:magn_encode(x))
df_all['RTDATE'] = df_all['RTDATE'].apply(lambda x:date_encode(x))
df_all['STDATE'] = df_all['STDATE'].apply(lambda x:date_encode(x))
df_all['OP'] = df_all['OP'].apply(lambda x:date_encode(x))
df_all['CIG'] = df_all['CIG'].apply(lambda x:cig_encode(x))
df_all['BN'] = df_all['BN'].apply(lambda x:bn_encode(x))
df_all['ALC'] = df_all['ALC'].apply(lambda x:alc_encode(x))
df_all['SSF1'] = df_all['SSF1'].apply(lambda x:ssf1_encode(x))
df_all['SSF2'] = df_all['SSF2'].apply(lambda x:ssf2_encode(x))
df_all['SSF3'] = df_all['SSF3'].apply(lambda x:ssf3_encode(x))
df_all['SSF4'] = df_all['SSF4'].apply(lambda x:ssf4_encode(x))
df_all['SSF6'] = df_all['SSF6'].apply(lambda x:ssf6_encode(x))
df_all['BMI_label'] = df_all.apply(lambda x:bmi(x), axis = 1)

### 新四家 直接用別人編好的編碼檔

In [6]:
# pd.set_option('display.max_columns', None)
df_new = pd.read_csv('/home/refu0917/lungcancer/server/5site_encode.csv')
df_new = df_new[df_new['醫院別'] != '長庚']
columns = ['醫院別', '有無二癌', '性別', '診斷年齡', '分級分化', '腫瘤大小', '區域淋巴結侵犯數目', '整併期別'
        , '原發部位手術邊緣', '放射治療', '全身性治療', '原發部位最確切的手術切除日期', 'BMI'
        , '吸菸行為' ,'嚼檳榔行為', '喝酒行為', 'SSF1_同側肺但非主病灶的另外肺腫瘤', 'SSF2_波及臟層膜 彈性層'
        , 'SSF3_首次治療前生活功能狀態的評估', 'SSF4_惡性肋膜積水', 'SSF6_EGFR 基因突變']

col_map = ["LOC", "Class", "Gender", "Age", "DIFF", "TMRSZ", "LYMND"
            , "AJCCstage", "MAGN", "RTDATE", "STDATE", "OP", "BMI_label"
            , "CIG", "BN", "ALC", "SSF1", "SSF2", "SSF3", "SSF4", "SSF6"    
            ]
df_new = df_new[columns]
df_new = df_new.rename(columns = { i:j for i, j in zip(columns, col_map)})
df_new.OP = df_new.OP.apply(lambda x:1 if x == 0 else 2)

### df_all和別人編好的　value count 有差異的columns (DIFF), BMi AJCCstage別人的編碼檔好像有問題

In [143]:
d = pd.DataFrame()
d['all'] = df_all.BMI_label.reset_index(drop=True)
d['new'] = df_new.BMI_label.reset_index(drop=True)
d['pid'] = df_all.id.reset_index(drop=True)

In [230]:
df_new.DIFF.value_counts()

d = pd.DataFrame()
d['臨床分級分化'] = df_all.DIFF_clin.reset_index(drop=True)
d['病理分級分化'] = df_all.DIFF_path.reset_index(drop=True)
d['分級分化'] = df_all.DIFF_check.reset_index(drop=True)
d['我編的分級分化'] = df_all.DIFF.reset_index(drop=True)
d['編碼檔分級分化'] = df_new.DIFF.reset_index(drop=True)
d['patient_id'] = df_all.id.reset_index(drop=True)


## 舊四家(長庚)

In [6]:
df_raw = pd.read_csv(r'/home/refu0917/lungcancer/server/AllCaseCtrl_RAW_Process.csv',index_col=[0])
df_final = pd.read_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_final.csv')
columns1 = ["Class","LOC", "FullDate","IDCODE","Gender", "Age", "CIG",
            "ALC", "BN", "MAGN", "AJCCstage", "DIFF", "LYMND",
            "TMRSZ", "OP", "RTDATE", "STDATE", "BMI_label",
            "SSF1", "SSF2", "SSF3", "SSF4", "SSF6"]
df_raw = df_raw[columns1]
df_final = df_final[columns1]
df_final['year'] = df_final['FullDate'].apply(lambda x:int(x[:4]))
df_final = df_final[df_final['year']>=2011]
df_final.isna().sum(axis=1).value_counts()
df_raw['null_count'] = df_raw.isna().sum(axis=1)

In [7]:
df = pd.read_csv('/home/refu0917/lungcancer/server/veryraw.csv')
df = df.loc[(df.CLASSOFCASE == 1) | (df.CLASSOFCASE == 2)]
df = df[df.CNRSEQ == 1] 
df.FullDate = df.FullDate.apply(lambda x :int(x[:4]))
df = df[df.FullDate > 2010]
columns2 = ['LOC', 'Class', 'Gender', 'IDCODE','FullDate', 'CNRDXAGE', 'HTLYDIFLEV', 'TMRSZ1', 'ARELYPHPOSNUM', 'DEFISURPRIDAT', 'TMRCUTMAGN'
        , 'CLNCMIPDCMB', 'PTHLMIPDCMB', 'INIDATEBEAMRADI', 'SYSTHERSTRDAT', 'WEIGHT', 'HIGHT', 'SMOK_1', 'SMOK_2', 'SMOK_3'
        ,'BETELNUT_1', 'BETELNUT_2', 'BETELNUT_3', 'DRINK', 'SSF1', 'SSF2', 'SSF3', 'SSF4', 'SSF6']
df = df[columns2]

In [8]:
def age_encode(age):
    if age < 21:
        return 1
    elif 21 <= age < 31:
        return 2
    elif 31 <= age < 41:
        return 3
    elif 41 <= age < 51:
        return 4
    elif 51 <= age < 61:
        return 5
    elif 61 <= age < 71:
        return 6
    elif 71 <= age < 81:
        return 7
    elif 81 <= age < 91:
        return 8
    elif age >= 91:
        return 9
def size_encode(size):
    if 0<=size<=49 or 991 <= size <= 995:
        return 1
    elif 50 <= size <= 99:
        return 2
    elif 100 <= size <= 149:
        return 3
    elif 150 <= size < 990:
        return 4
    elif size in [990,998,999]:
        return 9
    else:return np.nan
def lymnd_encode(x):
    if x == 0:
        return 1
    elif 1 <= x <= 2:
        return 2
    elif 3 <= x <= 6:
        return 3
    elif 7 <= x <= 15:
        return 4
    elif 16 <= x < 95:
        return 5
    elif x in [95,97,98,99]:
        return 9
def ajcc_encode(x):
    val = 0
    val = x['CLNCMIPDCMB']  if pd.isnull(x['DEFISURPRIDAT']) else x['PTHLMIPDCMB']
    
    '''if pd.isnull(x['DEFISURPRIDAT']) == True:
        val = x['CLNCMIPDCMB']  
    elif pd.isnull(x['DEFISURPRIDAT']) == False:
        val = x['PTHLMIPDCMB']'''
    
    if val in ['BBB', '888', '999', 'OC ']:
        return 9
    else:
        temp = int(val[0])
        if temp == 0:
            return 1
        elif temp == 1:
            return 2
        elif temp == 2:
            return 3
        elif temp == 3:
            return 4
        elif temp == 4:
            return 5


def magn_encode(x):
    if pd.isnull(x):
        return np.nan
    elif x in ['A','B','C']:
        return 2
    else:
        x = int(x)
        if x == 0:
            return 1
        elif 0 < x <7:
            return 2
        elif x in[7,8,9]:
            return 9
        else: return np.nan


def bmi(x):
    if pd.isnull(x['HIGHT']) or pd.isnull(x['WEIGHT']):
        return 9
    elif x['HIGHT'] == 999 or x['WEIGHT'] == 999:
        return 9
    else:
        h = x['HIGHT']
        h /= 100
        w = x['WEIGHT']
        output = w/(h**2)
        if output < 18.5:
            return 1
        elif 18.5 <= output < 24:
            return 2
        elif output >= 24:
            return 3

def ssf1_encode(x):
    if x == 0 or x == '000':
        return 1
    elif x in[10,20,30,40,'030','010','020','040']:
        return 2
    elif x in [999, 988,'988','999']:
        return 9
    else: return np.nan

def ssf2_encode(x):
    if x == 0 or x == '000':
        return 1
    elif x in[10,20,30,40,'010','020','030','040']:
        return 2
    elif x in [999, 988, '988', '999']:
        return 9
    else: return x

def ssf3_encode(x):
    if pd.isnull(x):
        return np.nan
    elif x in [999,988,998]:
        return 9
    
    else: return x+1 

def ssf4_encode(x):
    if x == 0:
        return 1
    elif 11<=x<=15:
        return 2
    elif x in [999,988]:
        return 9
    else: return np.nan

def ssf6_encode(x):
    if x == 20:
        return 1
    elif x ==10:
        return 2
    elif x in [0,30,999,998,988,2]:
        return 9
    else: return np.nan 

def diff_encode(x):
    if x == '6': return 9
    elif pd.isnull(x): return x
    else: return int(x)

def smok_encode(x):
    if pd.isnull(x['SMOK_3']) and pd.isnull(x['SMOK_2']) and pd.isnull(x['SMOK_1']):
        return np.nan
    elif x['SMOK_1'] in [99] and x['SMOK_2'] in [99] and x['SMOK_3'] in [99]:
        return 9
    elif  x['SMOK_1'] == 0 and x['SMOK_2'] == 0 and x['SMOK_3'] == 88:
        return 1
    else:
        return 2

def bn_encode(x):
    if pd.isnull(x['BETELNUT_1']) and pd.isnull(x['BETELNUT_2']) and pd.isnull(x['BETELNUT_3']):
        return np.nan
    elif x['BETELNUT_1'] in [99] and x['BETELNUT_2'] in [99] and x['BETELNUT_2'] in [99]:
        return 9
    elif  x['BETELNUT_1'] == 0 and x['BETELNUT_2'] == 0 and x['BETELNUT_3'] == 88:
        return 1
    else:
        return 2

def alc_encode(x):
    if x == 0:
        return 1
    elif 0 < x < 10:
        return 2
    elif x == 999:
        return 9
    else: return np.nan

In [9]:
df['Class'] = df['Class'].apply(lambda x:1 if x != 0 else x)
df['Gender'] = df['Gender'].apply(lambda x:1 if x == 'M' else 2)
df['Age'] = df['CNRDXAGE'].apply(lambda x:age_encode(x))
df['TMRSZ'] = df['TMRSZ1'].apply(lambda x:size_encode(x))
df['LYMND'] = df['ARELYPHPOSNUM'].apply(lambda x:lymnd_encode(x))
# df['AJCCstage'] = df.apply(lambda x:ajcc_encode(x), axis=1)
df['MAGN'] = df['TMRCUTMAGN'].apply(lambda x:magn_encode(x))
df['RTDATE'] = df['INIDATEBEAMRADI'].apply(lambda x:1 if pd.isnull(x) else 2)
df['STDATE'] = df['SYSTHERSTRDAT'].apply(lambda x:1 if pd.isnull(x) else 2)
df['OP'] = df['DEFISURPRIDAT'].apply(lambda x:1 if pd.isnull(x) else 2)
df['DIFF'] = df['HTLYDIFLEV'].apply(lambda x:diff_encode(x))
df['BMI_label'] = df.apply(lambda x:bmi(x), axis=1)
df['SSF1'] = df['SSF1'].apply(lambda x:ssf1_encode(x))
df['SSF2'] = df['SSF2'].apply(lambda x:ssf2_encode(x))
df['SSF3'] = df['SSF3'].apply(lambda x:ssf3_encode(x))
df['SSF4'] = df['SSF4'].apply(lambda x:ssf4_encode(x))
df['SSF6'] = df['SSF6'].apply(lambda x:ssf6_encode(x))
df['CIG'] = df.apply(lambda x:smok_encode(x), axis=1)
df['BN'] = df.apply(lambda x:bn_encode(x), axis=1)
df['ALC'] = df['DRINK'].apply(lambda x:alc_encode(x))
df = pd.merge(df, df_raw[['IDCODE','AJCCstage']],on = ['IDCODE'])

In [10]:
columns = ["Class","LOC", "FullDate","Gender", "Age", "CIG",
            "ALC", "BN", "MAGN", "AJCCstage", "DIFF", "LYMND",
            "TMRSZ", "OP", "RTDATE", "STDATE", "BMI_label",
            "SSF1", "SSF2", "SSF3", "SSF4", "SSF6"]
loc_dict = {'亞東':9, '中山':10, '中慈':11, '仁愛':12}

df_old = df[columns]
df_new = df_all[columns]
df_new = df_new.replace({"LOC":loc_dict})

In [11]:
df_old.to_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_4hospital_v3.csv')

In [12]:
from sklearn.utils import shuffle

df_new = pd.concat([df_old, df_new])
df_new = shuffle(df_new)
df_new.to_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_8hospital_v3.csv')

In [15]:
df_1 = pd.read_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_8hospital.csv',index_col=[0])
df_2 = pd.read_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_8hospital_v2.csv',index_col=[0])
df_3 = pd.read_csv('/home/refu0917/lungcancer/server/AllCaseCtrl_8hospital_v3.csv',index_col=[0])

In [19]:
df_1.OP.value_counts()

1    6474
0    3028
2    1934
Name: OP, dtype: int64

In [20]:
df_2.OP.value_counts()

1    8111
2    3325
Name: OP, dtype: int64

In [21]:
df_3.OP.value_counts()

1    8111
2    3325
Name: OP, dtype: int64